# OpenAI Function Calling


**Notes**:

- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.


### Get current weather


In [8]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']


In [9]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


In [10]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]


In [11]:
messages = [
  {
    "role": "user",
    "content": "What's the weather like in San Francisco?",
  }
]


In [12]:
import openai


In [13]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)


In [14]:
print(response)


{
  "id": "chatcmpl-8alGlu5KTkWb0ga4lPpu8YPz3eqLb",
  "object": "chat.completion",
  "created": 1703772203,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 19,
    "total_tokens": 102
  },
  "system_fingerprint": null
}


In [15]:
response_message = response['choices'][0]['message'] 


In [16]:
response_message


<OpenAIObject at 0x7f06d00ead50> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
  }
}

In [17]:
response_message["content"]


In [18]:
response_message["function_call"]


<OpenAIObject at 0x7f06d00eb350> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
}

In [19]:
json.loads(response_message["function_call"]["arguments"])


{'location': 'San Francisco, CA'}

In [20]:
args = json.loads(response_message["function_call"]["arguments"])


In [21]:
get_current_weather(args)


'{"location": {"location": "San Francisco, CA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

### Not using functions


In [22]:
messages = [
  {
    "role": "user",
    "content": "hi!",
  }
]


In [23]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)


In [24]:
print(response)


{
  "id": "chatcmpl-8alGm0QiUqY31BfhBsAR9Z92uxUgI",
  "object": "chat.completion",
  "created": 1703772204,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


## The `function_call` parameter


### Auto mode (the default mode)


In [25]:
messages = [
  {
    "role": "user",
    "content": "hi!",
  }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)


{
  "id": "chatcmpl-8alGn0OeR3eZdS9g8HsEEaeh5rtWG",
  "object": "chat.completion",
  "created": 1703772205,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


### Forcing NOT using functions


`content` will not matter because the `function_call` is set to `none`. This block will not return the `function_call` object from the OpenAI API.


In [26]:
messages = [
  {
    "role": "user",
    "content": "What's the weather like in San Francisco?",
  }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)


{
  "id": "chatcmpl-8alGohI5Bw7bvsbiG74SOonKBVTsd",
  "object": "chat.completion",
  "created": 1703772206,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Let me check the current weather in San Francisco for you."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 84,
    "completion_tokens": 12,
    "total_tokens": 96
  },
  "system_fingerprint": null
}


### Forcing using functions


In [27]:
messages = [
  {
    "role": "user",
    "content": "What's the weather like in San Francisco?",
  }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, # this will force it to use the get_current_weather function

)
print(response)


{
  "id": "chatcmpl-8alGpzuFK8NY7KVWdpk9CobzFxhL0",
  "object": "chat.completion",
  "created": 1703772207,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 90,
    "completion_tokens": 12,
    "total_tokens": 102
  },
  "system_fingerprint": null
}


However, passing a content that doesn't need a function will still return the `function_call` object from the OpenAI API.


In [28]:
messages = [
  {
    "role": "user",
    "content": "hi!",
  }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, # this will force it to use the get_current_weather function

)
print(response)


{
  "id": "chatcmpl-8alGqYg2rXSmhagR1itE0gDWmsZij",
  "object": "chat.completion",
  "created": 1703772208,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\",\n  \"unit\": \"celsius\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 20,
    "total_tokens": 103
  },
  "system_fingerprint": null
}


Now will multiple functions


In [29]:
# define a function
functions = [
    {
        "name": "weather_function",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
    ,{
        "name": "math_function",
        "description": "Perform a math operation",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "description": "The operation to perform, e.g. 2+2",
                },
            },
            "required": ["operation"],
        },

    }
]

messages = [
  {
    "role": "user",
    "content": "Square root of 456",
  }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto", # this will force it to use the get_current_weather function

)
print(response)


{
  "id": "chatcmpl-8alGrjKjQL9UNaUBcwIYoZgtmIZHN",
  "object": "chat.completion",
  "created": 1703772209,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "math_function",
          "arguments": "{\n  \"operation\": \"sqrt(456)\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 113,
    "completion_tokens": 17,
    "total_tokens": 130
  },
  "system_fingerprint": null
}


### Getting the result of the `function_call` object, using the correct function and passing back to the LLM


In [30]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]
messages = [
  {
    "role": "user",
    "content": "What's the weather like in San Francisco?",
  }
]
completions = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, # this will force it to use the get_current_weather function

)
response_message = completions.choices[0].message
messages.append(response_message)


In [31]:
function_args = json.loads(response_message.function_call.arguments)
observation = get_current_weather(function_args)


In [32]:
messages.append(
  {
    "role": "function",
    "name": "get_current_weather",
    "content": observation,
  }
)


Explanation:

- `"role": "function"`: This is to convey to the LLM that's the response of calling a function.
- `"function_name": "get_current_weather"`: This is the name of the function that was called.
- `"content"`: Set to `observation`, calculated above.


In [33]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-0613",
  messages=messages,
)
print(response)
print(messages)


{
  "id": "chatcmpl-8alGsJ1kXNKYFBEvmsLxRwCWOzWEf",
  "object": "chat.completion",
  "created": 1703772210,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in San Francisco, CA is 72\u00b0F with a sunny and windy forecast."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 19,
    "total_tokens": 98
  },
  "system_fingerprint": null
}
[{'role': 'user', 'content': "What's the weather like in San Francisco?"}, <OpenAIObject at 0x7f06d00ebb90> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"San Francisco, CA\"\n}"
  }
}, {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": {"location": "San Francisco, CA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "wind